In [1]:
# !pip install geopy
#!pip install gmaps

In [2]:
# importing libraries

import requests
import geopy
#import gmaps
from geopy.geocoders import Nominatim

In [3]:
# Get latitude,longitude for Origin and Destination

def location_converter(location):
    geolocator = Nominatim()
    location = geolocator.geocode(location)
    return (location.latitude, location.longitude)

# Ask for User Input for Origin and Destination
# origin = input('Origin: ').replace(' ','+')
# destination = input('Destination: ').replace(' ','+')

origin = "Vantaa"
destination = "Espoo"

#Getting the co-ordinates
origin_coordinates = location_converter(origin)
print("Origin: ", origin_coordinates)
destination_coordinates = location_converter(destination)
print("Destination: ", destination_coordinates)


/home/jamie/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


Origin:  (60.3091871, 25.0364526)
Destination:  (60.2051454, 24.6569676)


In [6]:
# A function to use queries to get data from HSL API 

def run_query(query, api_link): 
    request = requests.post(api_link, json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))

query = """
{
  
  plan(
    from: {lat: """+ str(origin_coordinates[0])+""", lon: """+str(origin_coordinates[1])+"""}
    to: {lat: """+ str(destination_coordinates[0])+""", lon: """+ str(destination_coordinates[1])+"""}
    numItineraries: 5
  ) {
    itineraries {
      legs {
        mode
        duration
        distance
      }}}}
"""

api_link = "https://api.digitransit.fi/routing/v1/routers/hsl/index/graphql"


# Execute the query and extract the result

result = run_query(query,api_link) 
result_dictionary = result["data"]["plan"]["itineraries"]

route_num = 0
for legs in result_dictionary:
    route_num += 1
    dict_split = dict(legs)
    dict_route = dict_split['legs']
    # print("Route Number: {} \n{}".format(route_num, dict_route))
    print("\n")
    
    duration = 0 
    distance = 0 
    carbon = 0
    
    for route in dict_route:
        print(route)
        
        #Calculating duration in min 
        duration += route['duration']
        duration_minute = int(duration/60)
        
        #Calculating distance in km 
        distance += route['distance']
        distance_km = round(float(distance/1000),2)
        
        carbon_factor = 0.065
        if route['mode'] in ("RAIL","TRAM","METRO"):
            carbon_factor = 0.065
        if route['mode'] == "BUS":
            carbon_factor = 0.069
        if route['mode'] in ("WALK"):
            carbon_factor = 0
        carbon += round(float((carbon_factor * distance_km)/1000),7)
        
    print("Total duration: {} min".format(duration_minute))
    print("Total distance: {} km".format(distance_km))
    print("Carbon emission: {} ton".format(carbon))



{'mode': 'WALK', 'duration': 246.0, 'distance': 294.316}
{'mode': 'BUS', 'duration': 120.0, 'distance': 1056.8089805928205}
{'mode': 'WALK', 'duration': 200.0, 'distance': 259.09700000000004}
{'mode': 'RAIL', 'duration': 1080.0, 'distance': 13760.3528036764}
{'mode': 'WALK', 'duration': 171.0, 'distance': 221.75900000000001}
{'mode': 'RAIL', 'duration': 1200.0, 'distance': 16910.53834756498}
{'mode': 'WALK', 'duration': 57.0, 'distance': 63.494}
Total duration: 51 min
Total distance: 32.57 km
Carbon emission: 0.0032048 ton


{'mode': 'WALK', 'duration': 1037.0, 'distance': 1252.2899999999993}
{'mode': 'RAIL', 'duration': 1901.0, 'distance': 27425.552394054142}
{'mode': 'RAIL', 'duration': 1020.0, 'distance': 13813.968448197931}
{'mode': 'WALK', 'duration': 57.0, 'distance': 63.494}
Total duration: 66 min
Total distance: 42.56 km
Carbon emission: 0.0046261 ton
